In [2]:
from google.cloud import storage
import os
import pandas as pd
import pandas_gbq

BUCKET_NAME = "datadrivers-demo"
BLOB_NAME = "BR_category_id.json"
PROJECT_ID = "fabled-archive-412122"
TABLE_NAME = "datadrivers_demo.ytcategories_trending"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/nguyehoa/Repositories/Personal/zoomcamp-homework/01-docker-terraform/keys/fabled-archive-412122-795c6304ed01.json"


storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob(BLOB_NAME)

with blob.open("r") as f:
    df = pd.read_json(f)

new_df = pd.json_normalize(df["items"])
new_df = new_df.rename(columns={"snippet.title": "snippet_title", "snippet.assignable": "snippet_assignable", "snippet.channelId": "snippet_channelId"})

pandas_gbq.to_gbq(new_df, TABLE_NAME, PROJECT_ID, location="europe-west3", if_exists="replace")
